In [1]:
"""
Correlation between FRP (intensity) and dNBR (severity)

maxwell.cook@colorado.edu
"""


import os, time, glob
import numpy as np

import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import rasterio as rio
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from rasterstats import zonal_stats

proj = 'ESRI:102039'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'

print("Complete")

Complete


In [2]:
# Functions

def list_dir(path):
    return glob.glob(os.path.join(path, '*'), recursive=True)

def list_files(path, ext, recursive):
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)

# Calculate zonal statistics
def calc_zonal_stats(vectors, raster_path, attr, stats=['mean'], categorical=False, geojson_out=True):
    begin = time.time()    

    # Calculate the zonal mean
    zs = zonal_stats(vectors=vectors[['VID','geometry']], raster=raster_path, stats=stats, categorical=categorical, geojson_out=geojson_out)
    
    # Convert the list of dicts to a DataFrame, extract properties
    zs_df = pd.DataFrame(zs)

    # Tidy the columns
    zs_df['VID'] = zs_df['properties'].apply(lambda x: x.get('VID'))
    zs_df['mean'] = zs_df['properties'].apply(lambda x: x.get('mean'))
    zs_df = zs_df[['VID','mean']]
    zs_df.rename(columns={'mean':f'{attr}_mean'}, inplace=True)

    print(zs_df.head())

    print(f"Total elapsed time for the current grid: {round((time.time() - begin))} seconds.")

    del zs
    
    return pd.DataFrame(zs_df)

print("Success!")

## Prep the burn severity data from MTBS

For not, using the MTBS burn severity. However, we want to use the composite burn index here eventually.

In [3]:
# Load the FRP observations
frp_plots = os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_plots_fired_events_west_aspen_LF.gpkg')
frp_plots = gpd.read_file(frp_plots)
frp_plots = frp_plots.to_crs(proj)
print(frp_plots.head())

      VID   LATITUDE   LONGITUDE   ACQ_DATE ACQ_TIME CONFIDENCE    FRP  \
0  876446  37.507347 -112.675858 2020-11-18     1954          n  24.28   
1  876447  37.502945 -112.674408 2020-11-18     1954          n  24.28   
2  876448  37.504166 -112.669052 2020-11-18     1954          n  24.28   
3  876763  37.512333 -112.681885 2020-11-19     1936          n   7.04   
4  876764  37.513638 -112.677109 2020-11-19     1936          n   7.04   

  DAYNIGHT fired_id              ig_date  ...      ig_utm_x      ig_utm_y  \
0        D    44927  2020-11-16 00:00:00  ...  4.167730e+06 -9.943850e+06   
1        D    44927  2020-11-16 00:00:00  ...  4.167730e+06 -9.943850e+06   
2        D    44927  2020-11-16 00:00:00  ...  4.167730e+06 -9.943850e+06   
3        D    44927  2020-11-16 00:00:00  ...  4.167730e+06 -9.943850e+06   
4        D    44927  2020-11-16 00:00:00  ...  4.167730e+06 -9.943850e+06   

      tot_perim pct_aspen  ACQ_MONTH  ACQ_YEAR  CBD_mean  CBH_mean  CC_mean  \
0  60253.2525

In [4]:
mtbs_dir = os.path.join(maindir,'data/mtbs/MTBS_BSmosaics')

results = []
print(frp_plots.columns)
# Loop the directories, calcl
for dir in list_dir(mtbs_dir):
    grid = list_files(dir, '.tif', recursive=True)[0]
    print(f'Starting for year {os.path.basename(dir)}')
    # extract the FRP obs. for that fire year
    ig_year = int(os.path.basename(dir))
    frp_year = frp_plots[frp_plots['ig_year'] == ig_year]
    # Calculate the zonal statistics
    df = calc_zonal_stats(vectors=frp_year[['VID','geometry']], raster_path=grid, attr='dnbr')
    print((df['dnbr_mean'].isna().sum() / len(df))*100)
    results.append(df)

# Concatenate the results
results = pd.concat(results)

# Merge back to the FRP obs.
frp_sev = frp_plots.merge(results, left_on='VID', right_on='VID', how='left')
print(frp_sev.head())

Index(['VID', 'LATITUDE', 'LONGITUDE', 'ACQ_DATE', 'ACQ_TIME', 'CONFIDENCE',
       'FRP', 'DAYNIGHT', 'fired_id', 'ig_date', 'ig_day', 'ig_month',
       'ig_year', 'last_date', 'event_dur', 'tot_ar_km2', 'fsr_km2_dy',
       'mx_grw_km2', 'mn_grw_km2', 'mu_grw_km2', 'mx_grw_dte', 'ig_utm_x',
       'ig_utm_y', 'tot_perim', 'pct_aspen', 'ACQ_MONTH', 'ACQ_YEAR',
       'CBD_mean', 'CBH_mean', 'CC_mean', 'geometry'],
      dtype='object')
Starting for year 2022
       VID dnbr_mean
0  1462030      None
1  1462031      None
2  1462032      None
3  1462033      None
4  1462036      None
Total elapsed time for the current grid: 5 seconds.
100.0
Starting for year 2019
      VID  dnbr_mean
0  422425   2.932800
1  422427   2.790816
2  422474   2.539749
3  422488   2.874439
4  422489   3.089385
Total elapsed time for the current grid: 4 seconds.
3.4982508745627188
Starting for year 2021
      VID  dnbr_mean
0  915953   2.869792
1  915954   2.180049
2  915956   2.499200
3  915958   2.388800
4  

/var/folders/lm/1zg27k9x385csjv9gkj8jqm80000gp/T/ipykernel_1308/4271804869.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat(results)
